### Special Thanks

🏀🏀 2022 March Mania - Quick EDA & FE🏀🏀
https://www.kaggle.com/kalilurrahman/2022-march-mania-quick-eda-fe

### If you copy or like this notebook, Please upvote !!

In [ ]:
!pip install tubesml==0.2.0

In [ ]:
import numpy as np 
import pandas as pd 

import tubesml as tml

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import mm_data_manipulation as dm

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Team

In [ ]:
df = pd.read_csv('../input/mens-march-mania-2022/MDataFiles_Stage1/MTeams.csv')
print(f'Shape: {df.shape}')
df.head()

In [ ]:
df['years_in_d1'] = df['LastD1Season'] - df['FirstD1Season']

df['years_in_d1'].hist(bins=20, figsize=(12,8))
plt.grid(False)
plt.title('Number of Years in Division 1', fontsize=16)
plt.show()

In [ ]:
df.FirstD1Season.hist(bins=20, alpha=0.7, label='First Season', figsize=(12,8))
df.LastD1Season.hist(bins=20, alpha=0.7, label='Last Season', figsize=(12,8))
plt.grid(False)
plt.legend()
plt.title('Distribution of First and Last Season in D1', fontsize=16)
plt.show()

## rating data

In [ ]:
rate_df = pd.read_csv('../input/ncaa-men-538-team-ratings/538ratingsMen.csv')
print(f'Shape: {df.shape}')
rate_df.head()

In [ ]:
rate_df["538rating"].min()

In [ ]:
len(rate_df[rate_df.Season == 2016])

In [ ]:
# rate_df['538rating'] = df.WScore - df.LScore
rate_df['538rating'].hist(bins=50, figsize=(12,8))
plt.grid(False)
plt.title('Rating in the regular season', fontsize=16)
plt.show()

In [ ]:
rate_df.describe()

In [ ]:
summaries = rate_df[['Season', 
                '538rating']].groupby('Season').agg(['min', 'max', 'mean', 'median'])

summaries.columns = ['_'.join(col).strip() for col in summaries.columns.values]
summaries

In [ ]:
summaries[[col for col in summaries.columns if '538rating' in col and 'sum' not in col]].plot(figsize=(12,8))
plt.title('Rating over time', fontsize=16)
plt.show()

- 実際のデータとレーティングの相関関係をみる
- レーティング高いチームの特徴を述べる
- レーティング高いチームはトーナメントに出場しているのか？？
- トップ10ぐらいを観察して傾向だしたい。何を出す必要ある
- 最近

## ordinal rank

In [ ]:
df = pd.read_csv('../input/mens-march-mania-2022/MDataFiles_Stage1/MMasseyOrdinals.csv')
print(f'Shape: {df.shape}')
df.head()

In [ ]:
df = df[df["Season"] >= 2015]

In [ ]:
# rate_df['538rating'] = df.WScore - df.LScore
df['OrdinalRank'].hist(bins=50, figsize=(12,8))
plt.grid(False)
plt.title('Rating in the regular season', fontsize=16)
plt.show()

In [ ]:
df.describe()

In [ ]:
summaries = df[['Season', 'TeamID',
                'OrdinalRank']].groupby(['Season', 'TeamID']).agg(['min', 'max', 'mean', 'median'])

summaries.columns = ['_'.join(col).strip() for col in summaries.columns.values]
summaries

In [ ]:
# summaries = df[['Season', 'TeamID',
#                 'OrdinalRank']].groupby(['Season', 'TeamID']).apply(lambda d: 100 if d.mean() >= 120 else 0)
summaries = df[['Season', 'TeamID',
                'OrdinalRank']].groupby(['Season', 'TeamID']).agg(['mean', 'median'])
summaries.columns = ['_'.join(col).strip() for col in summaries.columns.values]
summaries['OrdinalRank_mean'] = summaries['OrdinalRank_mean'].apply(lambda d: d if d <= 120 else None)
# summaries.columns = ['_'.join(col).strip() for col in summaries.columns.values]
summaries

In [ ]:
dff = summaries.reset_index()
dff

In [ ]:
dir_path = '../input/mens-march-mania-2022'
data_head = 'M'
tourney_results = pd.read_csv(f'{dir_path}/{data_head}DataFiles_Stage1/{data_head}NCAATourneyDetailedResults.csv')
# ../input/mens-march-mania-2022/MDataFiles_Stage1/MNCAATourneyDetailedResults.csv
seeds = pd.read_csv(f'{dir_path}/{data_head}DataFiles_Stage1/{data_head}NCAATourneySeeds.csv')
regular_results = pd.read_csv(f'{dir_path}/{data_head}DataFiles_Stage1/{data_head}RegularSeasonDetailedResults.csv')

def preprocess_csv(regular_results:pd.DataFrame):
    regular_results['WEFFG'] = regular_results['WFGM'] / regular_results['WFGA']
    # 2pt goal
    regular_results['WFGM2'] = regular_results['WFGM'] - regular_results['WFGM3']
    regular_results['WFGA2'] = regular_results['WFGA'] - regular_results['WFGA3']
    # success rate
    regular_results['WEFFG1'] = regular_results['WFTM'] / regular_results['WFTA']
    regular_results['WEFFG2'] = regular_results['WFGM2'] / regular_results['WFGA2']
    regular_results['WEFFG3'] = regular_results['WFGM3'] / regular_results['WFGA3']

    regular_results['WDARE'] = regular_results['WFGM3'] / regular_results['WFGM']
    regular_results['WTOQUETOQUE'] = regular_results['WAst'] / regular_results['WFGM']

    regular_results['LEFFG'] = regular_results['LFGM'] / regular_results['LFGA']
    # 2pt goal
    regular_results['LFGM2'] = regular_results['LFGM'] - regular_results['LFGM3']
    regular_results['LFGA2'] = regular_results['LFGA'] - regular_results['LFGA3']
    # success rate
    regular_results['LEFFG1'] = regular_results['LFTM'] / regular_results['LFTA']
    regular_results['LEFFG2'] = regular_results['LFGM2'] / regular_results['LFGA2']
    regular_results['LEFFG3'] = regular_results['LFGM3'] / regular_results['LFGA3']

    regular_results['LDARE'] = regular_results['LFGM3'] / regular_results['LFGM']
    regular_results['LTOQUETOQUE'] = regular_results['LAst'] / regular_results['LFGM']
    return regular_results

# tourney_results.Season.unique()
columns_list = ['Season', 'DayNum', 'T1_TeamID', 'T1_Score', 'T2_TeamID', 'T2_Score',
       'location', 'NumOT', 'T1_FGM', 'T1_FGA', 'T1_FGM3', 'T1_FGA3', 'T1_FGM2', 'T1_FGA2','T1_FTM',
       'T1_FTA', 'T1_OR', 'T1_DR', 'T1_Ast', 'T1_TO', 'T1_Stl', 'T1_Blk',
       'T1_PF', 'T2_FGM', 'T2_FGA', 'T2_FGM3', 'T2_FGA3', 'T2_FGM2', 'T2_FGA2','T2_FTM', 'T2_FTA',
       'T2_OR', 'T2_DR', 'T2_Ast', 'T2_TO', 'T2_Stl', 'T2_Blk', 'T2_PF',
       'T1_EFFG', 'T1_EFFG2', 'T1_EFFG3', 'T1_DARE',
       'T1_TOQUETOQUE', 'T2_EFFG','T2_EFFG2', 'T2_EFFG3', 'T2_DARE',
       'T2_TOQUETOQUE', 'PointDiff']

def prepare_data(df):
    dfswap = df[['Season', 'DayNum', 'LTeamID', 'LScore', 'WTeamID', 'WScore', 'WLoc', 'NumOT', 
    'LFGM', 'LFGA', 'LFGM3', 'LFGA3','LFGM2', 'LFGA2', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 
    'WFGM', 'WFGA', 'WFGM3', 'WFGA3','WFGM2', 'WFGA2','WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']]

    dfswap.loc[df['WLoc'] == 'H', 'WLoc'] = 'A'
    dfswap.loc[df['WLoc'] == 'A', 'WLoc'] = 'H'
    df.columns.values[6] = 'location'
    dfswap.columns.values[6] = 'location'    
      
    df.columns = [x.replace('W','T1_').replace('L','T2_') for x in list(df.columns)]
    dfswap.columns = [x.replace('L','T1_').replace('W','T2_') for x in list(dfswap.columns)]

    output = pd.concat([df, dfswap]).reset_index(drop=True)
    output.loc[output.location=='N','location'] = '0'
    output.loc[output.location=='H','location'] = '1'
    output.loc[output.location=='A','location'] = '-1'
    output.location = output.location.astype(int)
    
    output['PointDiff'] = output['T1_Score'] - output['T2_Score']
    output.reindex(columns=columns_list)
    return output

In [ ]:
regular_results = regular_results[regular_results["Season"] >= 2016]
regular_results.head()

In [ ]:
regular_results = preprocess_csv(regular_results)
regular_data = prepare_data(regular_results)

In [ ]:
regular_data

In [ ]:
dff_1 = dff.rename(columns={'TeamID':'T1_TeamID'})
dff_1.head()

In [ ]:
merge_df = regular_data.merge(dff_1, on=['T1_TeamID', 'Season'])
merge_df

In [ ]:
merge_df = merge_df[merge_df["OrdinalRank_mean"] <= 120]
merge_df

In [ ]:
summaries[[col for col in summaries.columns if 'OrdinalRank' in col and 'sum' not in col]].plot(figsize=(12,8))
plt.title('Rating over time', fontsize=16)
plt.show()

## ratingがどういう影響を与えるか観察する

In [ ]:
rate_df = pd.read_csv('../input/ncaa-men-538-team-ratings/538ratingsMen.csv')
print(f'Shape: {df.shape}')
rate_df.head()

In [ ]:
rate_df[rate_df['538rating'] == rate_df['538rating'].max()]

In [ ]:
no1_team = rate_df[rate_df.TeamID == 1211]
# no1_team = no1_team[['Season','538rating']].groupby("Season").agg(['min', 'max', 'mean', 'median'])
no1_team

## tourney

In [ ]:
tor_df = pd.read_csv('../input/mens-march-mania-2022/MDataFiles_Stage1/MNCAATourneyDetailedResults.csv')

In [ ]:
tor_df.head()

In [ ]:
r1_df = rate_df.copy()
r1_df["WTeamID"] = r1_df["TeamID"]
r1_df["rate_1"] = r1_df["538rating"]
# r_df["team2"] = r_df["TeamID"]
r1_df.head(20)

In [ ]:
r2_df = rate_df.copy()
r2_df["LTeamID"] = r2_df["TeamID"]
r2_df["rate_2"] = r2_df["538rating"]
# r_df["team2"] = r_df["TeamID"]
r2_df.head(20)

In [ ]:
tor_df.columns

In [ ]:
df = tor_df.copy()
df = df.astype(str)
r1_df = r1_df.astype(str)
r2_df = r2_df.astype(str)

# merge_df = pd.merge(sub_df, r_df, on=['team1', 'Season'])
# merge_df = pd.merge(sub_df, r_df, on=['team1', 'Season'])
merge_df = df.merge(r1_df, on=['WTeamID', 'Season'])
merge_df = merge_df.merge(r2_df, on=['LTeamID', 'Season'])

merge_df = merge_df[['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 'rate_1', 'rate_2']]
merge_df.head()

In [ ]:
merge_df['rate_diff'] = merge_df.rate_1.astype(float) - merge_df.rate_2.astype(float)
merge_df['point_diff'] = merge_df.WScore.astype(float) - merge_df.LScore.astype(float)
merge_df.head()

In [ ]:
merge_df['rate_1'].hist(bins=50, figsize=(12,8))
plt.grid(False)
plt.title('Rating difference in the tourney season', fontsize=16)
plt.show()

In [ ]:
merge_df['rate_2'].hist(bins=50, figsize=(12,8))
plt.grid(False)
plt.title('Rating difference in the tourney season', fontsize=16)
plt.show()

In [ ]:
merge_df['rate_diff'].hist(bins=50, figsize=(12,8))
plt.grid(False)
plt.title('Rating difference in the tourney season', fontsize=16)
plt.show()

In [ ]:
merge_df["rate_1"] = merge_df["rate_1"].astype(float)
merge_df["rate_2"] = merge_df["rate_2"].astype(float)

In [ ]:
merge_df.corr()

In [ ]:
sc = merge_df[['rate_1', 'rate_2','rate_diff', 'point_diff']]
figure = sns.pairplot(sc, plot_kws = {'alpha': 0.5})
figure.fig.set_figheight(10)
figure.fig.set_figwidth(10)

- 得点差とレート差にはやや正の相関見られる
- 実際に、レートと率の関係を考えてみる

## submit_df

In [ ]:
sub_df = pd.read_csv(f'../input/mens-march-mania-2022/MDataFiles_Stage1/MSampleSubmissionStage1.csv')

In [ ]:
sub_df.head()

In [ ]:
def id_0_apply(row):
    id = row["ID"]
    season, t1, t2 = id.split("_")
    return t1

def id_1_apply(row):
    id = row["ID"]
    season, t1, t2 = id.split("_")
    return t2

def season_apply(row):
    id = row["ID"]
    season, t1, t2 = id.split("_")
    return season

In [ ]:
df = sub_df.copy()
df["Season"]= df.apply(season_apply, axis=1)
df["team1"]= df.apply(id_0_apply, axis=1)
df["team2"]= df.apply(id_1_apply, axis=1)
df.head()

In [ ]:
r1_df = rate_df.copy()
r1_df["team1"] = r1_df["TeamID"]
r1_df["rate_1"] = r1_df["538rating"]
# r_df["team2"] = r_df["TeamID"]
r1_df.head(20)

In [ ]:
r2_df = rate_df.copy()
r2_df["team2"] = r2_df["TeamID"]
r2_df["rate_2"] = r2_df["538rating"]
# r_df["team2"] = r_df["TeamID"]
r2_df.head(20)

In [ ]:
df

In [ ]:
df.head()

In [ ]:
df = df.astype(str)
r1_df = r1_df.astype(str)
r2_df = r2_df.astype(str)

# merge_df = pd.merge(sub_df, r_df, on=['team1', 'Season'])
# merge_df = pd.merge(sub_df, r_df, on=['team1', 'Season'])
merge_df = df.merge(r1_df, on=['team1', 'Season'])
merge_df = merge_df.merge(r2_df, on=['team2', 'Season'])

merge_df = merge_df[['Pred','Season','team1','rate_1','team2','rate_2']]
merge_df.head()

In [ ]:
# merge_df["rate_diff"] = float(merge_df.rate_1) - float(merge_df.rate_2)
merge_df["rate_diff"] = merge_df.rate_1.astype(float) - merge_df.rate_2.astype(float)
merge_df